# Introduction

THis notebook contains he final notebook submission to the Kaggle competion predcit Calories challenge. I will start by first havoing code to the best possible solution based on the public leaderboard) and then explore other options. EDA and other stuff have been removed from this notebook for readability.
The code is inspired by this notebook.

https://www.kaggle.com/code/tmtngtrng/neural-network-and-lightgbm

In [1]:
# imports

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from itertools import combinations
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from IPython.display import clear_output
import warnings
warnings.simplefilter('ignore')

In [2]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')

In [3]:
train['bmi']=train['Weight']/(train['Height']*train['Height'])*100*100
test['bmi']=test['Weight']/(test['Height']*test['Height'])*100*100

train['dur_body_heart']=train['Duration']*train['Heart_Rate']*train['Body_Temp']
test['dur_body_heart']=test['Duration']*test['Heart_Rate']*test['Body_Temp']

def calculate_bmr(row):
    if row['Sex'].lower() == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    elif row['Sex'].lower() == 'female':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    else:
        return np.nan  # handle unknown or missing Sex

train['BMR'] = train.apply(calculate_bmr, axis=1)
test['BMR'] = test.apply(calculate_bmr, axis=1)

# Squares of features
train['dur_2']=train['Duration']*train['Duration']
test['dur_2']=test['Duration']*test['Duration']

train['hr_2']=train['Heart_Rate']*train['Heart_Rate']
test['hr_2']=test['Heart_Rate']*test['Heart_Rate']

train['height_2']=train['Height']*train['Height']
test['height_2']=test['Height']*test['Height']

train['weight_2']=train['Weight']*train['Weight']
test['weight_2']=test['Weight']*test['Weight']

# Square root of Duration
train['dur_sqrt'] = train['Duration'] ** 0.5
test['dur_sqrt'] = test['Duration'] ** 0.5

train['hr_sqrt'] = train['Heart_Rate'] ** 0.5
test['hr_sqrt'] = test['Heart_Rate'] ** 0.5

train['weight_sqrt'] = train['Weight'] ** 0.5
test['weight_sqrt'] = test['Weight'] ** 0.5

train['height_sqrt'] = train['Height'] ** 0.5
test['height_sqrt'] = test['Height'] ** 0.5

train['Intensity'] = train['Heart_Rate'] / train['Duration']
test['Intensity'] = test['Heart_Rate'] / test['Duration']

# Create binary indicators
train['is_male'] = (train['Sex'].str.lower() == 'male').astype(int)
train['is_female'] = (train['Sex'].str.lower() == 'female').astype(int)

test['is_male'] = (test['Sex'].str.lower() == 'male').astype(int)
test['is_female'] = (test['Sex'].str.lower() == 'female').astype(int)

# Numerical columns to interact with sex
num_cols = ['Age', 'Duration', 'Heart_Rate', 'Weight', 'Height', 'Body_Temp','bmi','Intensity','dur_body_heart']




for col in ['Weight', 'Height', 'Duration', 'Heart_Rate', 'Body_Temp']:
    train[f'{col}_log'] = np.log1p(train[col])
    test[f'{col}_log'] = np.log1p(test[col])

train['Weight_Height_ratio'] = train['Weight'] / train['Height']
test['Weight_Height_ratio'] = test['Weight'] / test['Height']

train['HeartRate_BodyTemp_ratio'] = train['Heart_Rate'] / train['Body_Temp']
test['HeartRate_BodyTemp_ratio'] = test['Heart_Rate'] / test['Body_Temp']


train['effort_per_kg'] = train['Duration'] * train['Heart_Rate'] / train['Weight']
test['effort_per_kg'] = test['Duration'] * test['Heart_Rate'] / test['Weight']


train['Age_HeartRate'] = train['Age'] * train['Heart_Rate']
test['Age_HeartRate'] = test['Age'] * test['Heart_Rate']

train['Temp_Height'] = train['Body_Temp'] * train['Height']
test['Temp_Height'] = test['Body_Temp'] * test['Height']

from itertools import combinations

poly_cols = ['Age', 'Duration', 'Heart_Rate', 'Weight', 'Height']
for col1, col2 in combinations(poly_cols, 2):
    train[f'{col1}_x_{col2}'] = train[col1] * train[col2]
    test[f'{col1}_x_{col2}'] = test[col1] * test[col2]


for col in poly_cols:
    col_mean = train[col].mean()  # Calculate mean from training data
    train[f'{col}_mean_diff'] = train[col] - col_mean
    test[f'{col}_mean_diff'] = test[col] - col_mean

In [9]:
train.columns

Index(['id', 'Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate',
       'Body_Temp', 'Calories', 'bmi', 'dur_body_heart', 'BMR', 'dur_2',
       'hr_2', 'height_2', 'weight_2', 'dur_sqrt', 'hr_sqrt', 'weight_sqrt',
       'height_sqrt', 'Intensity', 'is_male', 'is_female', 'Age_Male',
       'Age_Female', 'Duration_Male', 'Duration_Female', 'Heart_Rate_Male',
       'Heart_Rate_Female', 'Weight_Male', 'Weight_Female', 'Height_Male',
       'Height_Female', 'Body_Temp_Male', 'Body_Temp_Female', 'bmi_Male',
       'bmi_Female', 'Intensity_Male', 'Intensity_Female',
       'dur_body_heart_Male', 'dur_body_heart_Female', 'Weight_log',
       'Height_log', 'Duration_log', 'Heart_Rate_log', 'Body_Temp_log',
       'Weight_Height_ratio', 'HeartRate_BodyTemp_ratio', 'effort_per_kg',
       'Age_HeartRate', 'Temp_Height', 'Age_x_Duration', 'Age_x_Heart_Rate',
       'Age_x_Weight', 'Age_x_Height', 'Duration_x_Heart_Rate',
       'Duration_x_Weight', 'Duration_x_Height', 'Heart_Rate_x_We

In [4]:
def rmsle_loss(y_pred, y_true):
    y_pred = torch.clamp(y_pred, min=0)
    y_true = torch.clamp(y_true, min=0)
    return torch.sqrt(F.mse_loss(torch.log1p(y_pred), torch.log1p(y_true)))

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [6]:
class CalorieNet(nn.Module):
    def __init__(self, input_dim, dropout=0.1, use_residual=True):
            super(CalorieNet, self).__init__()
            
            self.use_residual = use_residual
            
            self.input_norm = nn.BatchNorm1d(input_dim)
            self.backbone = nn.ModuleList([
                self._make_block(input_dim, 512, dropout),
                self._make_block(512, 256, dropout),
                self._make_block(256, 128, dropout),
                self._make_block(128, 64, dropout),
            ])
            
            self.output_head = nn.Sequential(
                nn.Linear(64, 32),
                nn.GELU(),
                nn.Dropout(dropout * 0.5),  
                nn.Linear(32, 1),
                nn.Softplus()
            )
            

            self._initialize_weights()
        
    def _make_block(self, in_features, out_features, dropout):
        return nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            nn.GELU(),
            nn.Dropout(dropout)
        )
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.input_norm(x)
        for block in self.backbone:
            if self.use_residual and x.shape[1] == block[0].in_features and x.shape[1] == block[0].out_features:
                x = x + block(x)
            else:
                x = block(x)
        x = self.output_head(x)
        return x

In [7]:
drop_cols = ['id', 'Calories']
train['Sex'] = train['Sex'].map({'male': 1, 'female': 0})
test['Sex'] = test['Sex'].map({'male': 1, 'female': 0})
X = train.drop(columns=drop_cols)
y = train['Calories']


def get_max_min(series: pd.Series) -> list:
    return [series.max(), series.min()]

def max_min_scalse(series: pd.Series) -> pd.Series:
    max_min = get_max_min(series)
    return series.apply(lambda x: (x - max_min[1]) / (max_min[0] - max_min[1]))

for col in X.columns:
    if col != 'Sex':
        X[col] = max_min_scalse(X[col])

In [8]:

batch_size = 512

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_test_tensor, y_test_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [9]:
device

device(type='cpu')

In [10]:
X_train.shape

(600000, 46)

In [11]:
model = CalorieNet(
        input_dim=46,  
        dropout=0.00,
        use_residual=True
    ).to(device)
no_decay = ['bias', 'LayerNorm.weight', 'BatchNorm']
decay_params = []
no_decay_params = []

for name, param in model.named_parameters():
    if not param.requires_grad:
        continue
    if any(nd in name for nd in no_decay):
        no_decay_params.append(param)
    else:
        decay_params.append(param)
        
optimizer_params = [
    {'params': decay_params, 'weight_decay': 5e-4}, 
    {'params': no_decay_params, 'weight_decay': 0.0}
]
loss_fn_rmsle = rmsle_loss
best_val_loss = float("inf")
patience = 25
counter = 0
scaler = torch.amp.GradScaler() if torch.cuda.is_available() else None
max_epochs = 500

optimizer = torch.optim.AdamW(
    optimizer_params,
    lr=3e-4, 
    betas=(0.9, 0.999),
    eps=1e-8,
    amsgrad=True
)

total_steps = max_epochs * len(train_loader)




scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-3,
    epochs=max_epochs,
    steps_per_epoch=len(train_loader),
    pct_start=0.1,  
    anneal_strategy='cos',
    div_factor=10.0,
    final_div_factor=100.0
)

for epoch in range(max_epochs):
    model.train()
    total_train_loss = 0
    total_train_rmsle = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{max_epochs}")

    for batch_X, batch_y in progress_bar:
        batch_X = batch_X.to(device, non_blocking=True)
        batch_y = batch_y.to(device, non_blocking=True)

        if scaler:
            with torch.amp.autocast('cuda'):
                output = model(batch_X)
                loss = rmsle_loss(output, batch_y)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            scaler.step(optimizer)
            scaler.update()
        else:
            output = model(batch_X)
            loss = rmsle_loss(output, batch_y)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()

        scheduler.step()  

        
        with torch.no_grad():
            rmsle = loss_fn_rmsle(output, batch_y)
            total_train_rmsle += rmsle.item()

        total_train_loss += loss.item()

        progress_bar.set_postfix(
            loss=loss.item(),
            rmsle=rmsle.item(),
            lr=scheduler.get_last_lr()[0]
        )

    avg_train_loss = total_train_loss / len(train_loader)
    avg_train_rmsle = total_train_rmsle / len(train_loader)

    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device, non_blocking=True)
            batch_y = batch_y.to(device, non_blocking=True)

            output = model(batch_X)
            val_loss = loss_fn_rmsle(output, batch_y)
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)

    print(f"\nEpoch {epoch+1}/{max_epochs}")
    print(f"Train Loss: {avg_train_loss:.6f}, Train RMSLE: {avg_train_rmsle:.6f}")
    print(f"Val RMSLE: {avg_val_loss:.6f}, LR: {scheduler.get_last_lr()[0]:.8f}")

    if avg_val_loss < best_val_loss:
        improvement = (best_val_loss - avg_val_loss) / best_val_loss * 100
        best_val_loss = avg_val_loss
        counter = 0
        torch.save(model.state_dict(), "calorie_model_best.pt")
        clear_output(wait=True)
        print(f"Saved best model (RMSLE: {best_val_loss:.6f}, improved by {improvement:.2f}%)")
        
    else:
        counter += 1
        print(f"No improvement for {counter}/{patience} epochs")
        if counter >= patience:
            
            print(f"Early stopping triggered after {patience} epochs without improvement")
            print(f"Best RMSLE: {best_val_loss:.6f}")
            break

Saved best model (RMSLE: 0.059356, improved by 1.56%)


Epoch 30/500: 100%|████████████████████████████████████| 1172/1172 [00:08<00:00, 141.99it/s, loss=0.0652, lr=0.00207, rmsle=0.0652]



Epoch 30/500
Train Loss: 0.063547, Train RMSLE: 0.063547
Val RMSLE: 0.061244, LR: 0.00206721
No improvement for 1/25 epochs


Epoch 31/500: 100%|██████████████████████████████████████| 1172/1172 [00:08<00:00, 142.53it/s, loss=0.069, lr=0.00215, rmsle=0.069]



Epoch 31/500
Train Loss: 0.063741, Train RMSLE: 0.063741
Val RMSLE: 0.062841, LR: 0.00214701
No improvement for 2/25 epochs


...


Epoch 52/500
Train Loss: 0.061938, Train RMSLE: 0.061938
Val RMSLE: 0.059994, LR: 0.00299985
No improvement for 23/25 epochs


Epoch 53/500: 100%|██████████████████████████████████████| 1172/1172 [00:08<00:00, 145.01it/s, loss=0.0493, lr=0.003, rmsle=0.0493]



Epoch 53/500
Train Loss: 0.061919, Train RMSLE: 0.061919
Val RMSLE: 0.059985, LR: 0.00299967
No improvement for 24/25 epochs


Epoch 54/500: 100%|██████████████████████████████████████| 1172/1172 [00:07<00:00, 156.57it/s, loss=0.0567, lr=0.003, rmsle=0.0567]



Epoch 54/500
Train Loss: 0.061900, Train RMSLE: 0.061900
Val RMSLE: 0.060230, LR: 0.00299942
No improvement for 25/25 epochs
Early stopping triggered after 25 epochs without improvement
Best RMSLE: 0.059356


# Do predcitions

In [13]:
model = CalorieNet(
    input_dim=46,  
    dropout=0.00,
    use_residual=True
).to(device)

model.load_state_dict(torch.load("calorie_model_best.pt"))
model.eval()



CalorieNet(
  (input_norm): BatchNorm1d(46, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (backbone): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=46, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.0, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.0, inplace=False)
    )
    (2): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.0, inplace=False)
    )
    (3): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
   

In [36]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

# Drop the 'id' column to get model inputs
X_test = test.drop(columns=['id'])




for col in X_test.columns:
    if col != 'Sex':
        X_test[col] = max_min_scalse(X_test[col])


# Convert to tensor
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Create DataLoader
test_loader = DataLoader(X_test_tensor, batch_size=128, shuffle=False)

# Load model
model = CalorieNet(
    input_dim=59,  
    dropout=0.00,
    use_residual=True
).to(device)

model.load_state_dict(torch.load("calorie_model_best.pt"))
model.eval()

# Predict
all_preds = []

with torch.no_grad():
    for batch_X in test_loader:
        batch_X = batch_X.to(device, non_blocking=True)
        preds = model(batch_X)
        all_preds.append(preds.cpu())

# Combine and convert predictions
final_preds = torch.cat(all_preds, dim=0).numpy()

# Apply inverse log transform if necessary
final_preds = np.expm1(final_preds)

# Save predictions
submission_df = pd.DataFrame({
    'id': test['id'].values,
    'Calories': final_preds.flatten()
})
submission_df.to_csv('calorie_predictions.csv', index=False)

In [40]:

# Make sure test uses the same feature columns as X
X_test_new = test[X.columns]  # Ensure same order and columns


for col in X_test.columns:
    if col != 'Sex':
        X_test_new[col] = max_min_scalse(X_test_new[col])
        
# Convert to tensor
X_test_tensor = torch.tensor(X_test_new.values, dtype=torch.float32).to(device)

# DataLoader
test_loader = DataLoader(X_test_tensor, batch_size=512, shuffle=False)

# Recreate and load the trained model
model = CalorieNet(
    input_dim=X.shape[1],  # same as train
    dropout=0.00,
    use_residual=True
).to(device)

model.load_state_dict(torch.load("calorie_model_best.pt"))
model.eval()

# Predict
all_preds = []

with torch.no_grad():
    for batch_X in test_loader:
        batch_X = batch_X.to(device, non_blocking=True)
        preds = model(batch_X)
        all_preds.append(preds.cpu())

# Combine predictions
final_preds = torch.cat(all_preds, dim=0).numpy()

# Apply inverse log transform (since you trained on log1p)
final_preds = np.expm1(final_preds)
final_preds = np.clip(final_preds, 0, None)  # Remove negative/infinite values

# Save to CSV
submission_df = pd.DataFrame({
    'id': test['id'].values,
    'Calories': final_preds.flatten()
})
submission_df.to_csv('calorie_predictions.csv', index=False)

In [17]:

# Apply Min-Max scaling (same as in training)
def get_max_min(series: pd.Series) -> list:
    return [series.max(), series.min()]

def max_min_scale(series: pd.Series) -> pd.Series:
    max_min = get_max_min(series)
    return series.apply(lambda x: (x - max_min[1]) / (max_min[0] - max_min[1]))

for col in test.columns:
    if col not in ['id', 'Sex']:
        test[col] = max_min_scale(test[col])

# --- Tensor Conversion ---
X_final_test = test.drop(columns=['id'])
X_final_tensor = torch.tensor(X_final_test.values, dtype=torch.float32).to(device)

# --- Model Loading ---
model = CalorieNet(
    input_dim=46,  
    dropout=0.00,
    use_residual=True
).to(device)

model.load_state_dict(torch.load("calorie_model_best.pt"))
model.eval()

# --- Predictions ---
with torch.no_grad():
    preds = model(X_final_tensor).cpu().numpy().flatten()

# --- Save Results ---
test['Calories'] = preds
test[['id', 'Calories']].to_csv('calorie_predictions_new.csv', index=False)

In [15]:
len(test)

250000

In [16]:
test.shape

(250000, 47)